### GW tutorial 3: Generation of data and matched filtering

Author: Melissa Lopez

Email: m.lopez@uu.nl

In [ ]:
%matplotlib inline
from pycbc.catalog import Merger
import pycbc.psd, pycbc.noise
import pylab
import matplotlib.pyplot as plt
import numpy as np
from pycbc.waveform import get_td_waveform
from pycbc.detector import Detector
import random

Previously, we have plotted the PSD of different detectors. Some PSDs are from previous runs (O3), while some others are the design sensitivity of future detectors.

The PSD characterizes the noise of the detector, so we can generate detector noise accordingly.

**Exercise 1:** Load the `aligo_O4high.txt` PSD to [generate](https://pycbc.org/pycbc/latest/html/pycbc.noise.html#pycbc.noise.gaussian.noise_from_psd) some Gaussian data.

_Hint:_ Minimum frequency is 10 Hz, sampling rate 4096 Hz and we want 20s of duration

In [ ]:


# Load the PSD
psd = pycbc.psd.read.from_txt("aligo_O4high.txt")

# Generate the Gaussian noise
delta_f = 1.0 / 20
flen = int(2048 / delta_f) + 1
flow = 10
delta_t = 1.0/4096
tsamples = int(20/delta_t)
noise = pycbc.noise.gaussian.noise_from_psd(tsamples, delta_t, psd, seed=128)

# Plot the noise
pylab.plot(noise.sample_times, noise)
pylab.xlabel('Time (s)')
pylab.ylabel('Strain')
pylab.title('Gaussian Noise')
pylab.show()


As you can see. the generated noise is "coloured" according to the detector noise. Now, we would like to add a simulated gravitational wave signal in our detector noise.

**Exercise 2:** Using [this function](https://pycbc.org/pycbc/latest/html/pycbc.waveform.html#pycbc.waveform.waveform.get_td_waveform) to generate waveforms in time domain, provide the plus and cross polarization of a binary black hole merger of $m_{1} = m_{2} = 50$ at 2000 Mpc. Use the waveform approximant "IMRPhenomD". How does the waveform amplitude compare to the detector noise?

The waveform comes from the source, but it needs to be projected in the detector. [Project](https://pycbc.org/pycbc/latest/html/pycbc.detector.html#pycbc.detector.ground.Detector.project_wave) the waveform on LIGO Livingston (L1) at random sky-location and polarization. You can specify your detector with [this](https://pycbc.org/pycbc/latest/html/pycbc.detector.html#pycbc.detector.ground.Detector) function. How does the waveform change because of this projection?

_Hint:_ Right ascension range is $[0, 2\pi]$, declination is $[-2\pi, 2\pi]$ and polarization is $[0, 2\pi]$.

In [1]:

import numpy as np
# Generate the waveform
hp, hc = get_td_waveform(approximant="IMRPhenomD",
                         mass1=50,
                         mass2=50,
                         delta_t=delta_t,
                         f_lower=flow,
                         distance=2000)

# Plot the waveform
pylab.plot(hp.sample_times, hp, label='Plus Polarization')
pylab.plot(hp.sample_times, hc, label='Cross Polarization')
pylab.xlabel('Time (s)')
pylab.ylabel('Strain')
pylab.title('Binary Black Hole Merger Waveform')
pylab.legend()
pylab.show()

# Compare waveform amplitude to detector noise
print("Waveform amplitude:", max(abs(hp)))
print("Noise amplitude:", max(abs(noise)))


# Project the waveform onto the LIGO Livingston detector
# Generate random sky location and polarization
right_ascension = random.uniform(0, 2 * np.pi)
declination = random.uniform(-np.pi/2, np.pi/2)
polarization = random.uniform(0, 2 * np.pi)


# Specify the LIGO Livingston detector
det = Detector("L1")

# Project waveform
hp_projected = det.project_wave(hp, hc, right_ascension, declination, polarization)

# Plot the projected waveform
pylab.plot(hp_projected.sample_times, hp_projected)
pylab.xlabel('Time (s)')
pylab.ylabel('Strain')
pylab.title('Projected Waveform on LIGO Livingston')
pylab.show()


NameError: name 'get_td_waveform' is not defined

**Exercise 2:** From before our detector noise is 20s long. Add the GW into noise after 5s. We also need to get the first 4s to estimate the PSD in the next excercise. Make an overlaid plot with the total data, the portion of data where the GW is added and the data needed to estimate the PSD.

_Bonus:_ Can you also plot the GW signal?


In [ ]:
# prompt: From before our detector noise is 20s long. Add the GW into noise after 5s. We also need to get the first 4s to estimate the PSD in the next excercise. Make an overlaid plot with the total data, the portion of data where the GW is added and the data needed to estimate the PSD.
# Bonus: Can you also plot the GW signal?

import numpy as np
# Add the GW into noise after 5s
# get the first 4s to estimate the PSD
# Make an overlaid plot with the total data, the portion of data where the GW is added and the data needed to estimate the PSD
# Bonus: Plot the GW signal

# Time vector
time = np.arange(0, 20, delta_t)

# Find the index corresponding to 5 seconds
index_5s = np.argmin(np.abs(time - 5))

# Find the index corresponding to 4 seconds
index_4s = np.argmin(np.abs(time - 4))

# Add the projected waveform to the noise
signal = noise.copy()
signal[index_5s:index_5s + len(hp_projected)] += hp_projected

# Plot the results
pylab.figure(figsize=(10, 6))
pylab.plot(time, signal, label='Total Data (Noise + GW)')
pylab.plot(time[index_5s:index_5s + len(hp_projected)], signal[index_5s:index_5s + len(hp_projected)], label='GW Signal Added')
pylab.plot(time[:index_4s], signal[:index_4s], label='Data for PSD Estimation')
pylab.plot(hp_projected.sample_times, hp_projected, label='GW Signal', linestyle='--')
pylab.xlabel('Time (s)')
pylab.ylabel('Strain')
pylab.title('Simulated Detector Data with GW Signal')
pylab.legend()
pylab.grid(True)
pylab.show()


**Exercise 3:** In a proper search we do not really have the PSD handy. We want to estimate the PSD with Welch's method (see [here](https://ccrma.stanford.edu/~jos/sasp/Welch_s_Method.html) for details), but we want to use the 4s of the beginning where the GW is not present using [this function](https://pycbc.org/pycbc/latest/html/pycbc.psd.html#pycbc.psd.estimate.welch). Plot the estimated PSD. Note that before we used a dummy whitening, and this one is a bit better.

What is the $\Delta_f$ of the estimated PSD? What is $\Delta_f$ of the data we want to whiten to see the GW signal?

_Hint:_ `seg_stride (int)` is usually half of `seg_len`.


In [ ]:


# Estimate the PSD using Welch's method
seg_len = int(1 / delta_t)  # 1 second segment length
seg_stride = int(seg_len / 2)  # 50% overlap
estimated_psd = pycbc.psd.welch(noise_4s, seg_len=seg_len, seg_stride=seg_stride)

# Plot the estimated PSD
pylab.loglog(estimated_psd.sample_frequencies, estimated_psd)
pylab.xlabel('Frequency (Hz)')
pylab.ylabel('Strain^2/Hz')
pylab.title('Estimated PSD using Welch\'s method')
pylab.show()

# Calculate delta_f for the estimated PSD
delta_f_estimated_psd = estimated_psd.delta_f
print(f"Delta f of the estimated PSD: {delta_f_estimated_psd}")

# Delta f of the data we want to whiten
print(f"Delta f of the data to whiten: {delta_f}")


Now that we have the PSD we need to interpolate it to match our data and then limit the filter length of 1 / PSD. After this, we can directly use this PSD to filter the data. Since the data has been highpassed above 12 Hz, and will have low values below this we need to informat the function to not include frequencies below this frequency.

**Exercise 4:** Use the functions [interpolate](https://pycbc.org/pycbc/latest/html/pycbc.psd.html#pycbc.psd.estimate.interpolate) and [inverse_spectrum_truncation](https://pycbc.org/pycbc/latest/html/pycbc.psd.html#pycbc.psd.estimate.inverse_spectrum_truncation) to achieve a proper formatting of the PSD.

_Hint:_ `max_filter_len (int)` is $4 \times$ sampling_rate. Note that the original PSD has a minimum frequency of 12Hz.

In [ ]:
# prompt: Use the functions interpolate and inverse_spectrum_truncation to achieve a proper formatting of the PSD.
# Hint: max_filter_len (int) is 4× sampling_rate. Note that the original PSD has a minimum frequency of 12Hz.

# Interpolate the PSD to match the data
interpolated_psd = pycbc.psd.interpolate(estimated_psd, signal.delta_f)

# Apply inverse_spectrum_truncation
max_filter_len = int(4 * signal.sample_rate)
psd_for_filtering = pycbc.psd.inverse_spectrum_truncation(interpolated_psd,
                                                        max_filter_len,
                                                        low_frequency_cutoff=12)

# Now you can use psd_for_filtering for whitening or other signal processing tasks.
# For example, you can plot it:
pylab.loglog(psd_for_filtering.sample_frequencies, psd_for_filtering)
pylab.xlabel('Frequency (Hz)')
pylab.ylabel('Strain^2/Hz')
pylab.title('Formatted PSD for Filtering')
pylab.show()


Now that the PSD is ready, we can whiten the data. Before we used a PyCBC function, but mathematically this is defined as

\begin{equation}
\tilde{d_w}(f) = \tilde{d}(f)/S_{n}^{-1/2}(f)
\end{equation}
where $\tilde{d}$ and $\tilde{d_w}(f)$ are the Fourier transform of the coloured data and whitened data, respectively.

**Exercise 5:** Whiten the data using the interpolated PSD. Crop 5s at the beginning and the end to avoid border effects (_aliasing_) and bandpass it as in the previous exercise. Can you see the GW signal?

In [ ]:

# Whiten the data
whitened_data = signal.to_frequencyseries() / psd_for_filtering**0.5

# Crop the data
crop_len = int(5 / delta_t)
whitened_data = whitened_data.crop(crop_len, crop_len)

# Bandpass the data (same as before)
f_low = 30
f_high = 200
whitened_data = whitened_data.bandpass(f_low, f_high)

# Convert back to time domain
whitened_data_td = whitened_data.to_timeseries()


# Plot the whitened data
pylab.plot(whitened_data_td.sample_times, whitened_data_td)
pylab.xlabel('Time (s)')
pylab.ylabel('Strain')
pylab.title('Whitened Data')
pylab.show()


So far, we buried a GW signal  $h(t)$ in stationary and Gaussian noise $n(t)$ with zero mean, such that $s(t) = h(t) + n(t)$. Given the optimal filter  $K(t)$,

\begin{equation}
\label{eq:filteredsignal}
\hat{s} = \int_{-\infty}^{\infty}  K(t)s(t) dt = \int_{-\infty}^{\infty} \tilde{K}(f)^{*}\tilde{s}(f) df, \quad \text{ where }  \tilde{s}(f) = \int_{-\infty}^{\infty} s(t)e^{-2 \pi i ft}dt
\end{equation}
where $\hat{s}$ is the filtered value of $s(t)$, $^*$ represents the complex conjugate, and $\tilde{\cdot }\ $ the Fourier transform.

The detection statistic that is maximised by the optimal filter $K(t)$  will be the SNR, defined as $\rho = S/N$. $S$ is the expected value of $\hat{s}$ when $h(t) \neq 0$, while $N$ is the squared root of the noise variance when $h(t) =0$. It can be demonstrated that the optimal filter $K(t)$ is the model of the GW signal itself, known as _template_.

The fundamental modelled detection technique is called "matched filtering", since the filter function is chosen to "match" the signal we are looking for. We can write the SNR between an unknown time series $s(t)$ and the template $h_{m}$ as

\begin{equation}
\label{eq:wienerscalar}
\rho =  4 \text{Re} \int_{0}^{\infty} \frac{\tilde{s}^{*}(f)}{S_{n}(f)} \tilde{h_{m}}(f) df.
\end{equation}



**Exercise 6:** Use the [matched_filter](https://pycbc.org/pycbc/latest/html/pycbc.filter.html#pycbc.filter.matchedfilter.matched_filter) function to filter the coloured data with the template. Crop the SNR time series 5s at each side and plot it together with the whitened data. Where is the GW signal? Note that the y-axis of the whitened data is _amplitude_ and the y-axis of the SNR timeseries is _SNR_.

How much is the maximum of the absolute value of the SNR time series? If it is larger than 5 it will generate a "trigger" for further analysis. Will the trigger be generated?

Note that the parameter space of GW signals is vast, so to find these signals, we will need to create a template bank and do this process for thousands of templates. Then, this becomes a high-performance computing problem!

_Hint_: you need to [resize](https://pycbc.org/pycbc/latest/html/pycbc.types.html#pycbc.types.array.Array.resize) the template and use [cyclic_time_shift](https://pycbc.org/pycbc/latest/html/pycbc.types.html#pycbc.types.frequencyseries.FrequencySeries.cyclic_time_shift)

In [ ]:


from pycbc.filter import matched_filter
from pycbc.types import TimeSeries

# Resize the template to match the data length
hp.resize(len(signal))

# Perform matched filtering
snr = matched_filter(hp, signal,
                     psd=psd_for_filtering,
                     low_frequency_cutoff=f_low)

# Time shift the SNR time series to account for the time delay in the matched filter
snr = snr.cyclic_time_shift(hp.start_time)

# Crop the SNR time series
snr = snr.crop(5, 5)

# Plot the SNR time series along with the whitened data
pylab.figure(figsize=(10, 6))
pylab.plot(snr.sample_times, abs(snr), label='SNR')
pylab.plot(whitened_data_td.sample_times, whitened_data_td, label='Whitened Data')
pylab.xlabel('Time (s)')
pylab.ylabel('SNR / Strain')
pylab.title('Matched Filter SNR')
pylab.legend()
pylab.grid(True)
pylab.show()

# Find the maximum of the absolute value of the SNR time series
max_snr = max(abs(snr))
print(f"Maximum SNR: {max_snr}")

# Check if a trigger is generated
if max_snr > 5:
    print("Trigger generated!")
else:
    print("Trigger not generated.")


Good job arriving at the end of the tutorial! This was a small peak at GW data analysis that I hope you found interesting.

There is a bonus track exercise if you are bored, but we can also have a chat about some more GW data analysis if you prefer.